In [ ]:
from matplotlib import pyplot as plt
import numpy as np

from utils import VolumeFile
#from inc.python_image_utilities.image_util import *

In [ ]:
#!srun --pty curl -L  https://uab.box.com/s/xiurb4d624gfood3p8z5np1remu4rt4c --output HAEB100093L_TRIMMED.vol

In [ ]:
test_file = "HAEB100093L_TRIMMED.vol"
v = VolumeFile()
v.decode(test_file)
    
image = v.data
shape = image.shape

In [ ]:
shape

In [ ]:
import cv2
cv2.imshow('ImageWindow',image[:,:,1])

In [ ]:
a = image[:,:,:]

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(a, interpolation='nearest')
plt.show()

In [ ]:
v_small = resize(image, scale=0.1)

In [ ]:
position = round(shape[2] * 0.60)
image_slice = image[:, :, position]
plt.imshow(clahe(image_slice.squeeze(), tile_size=(50,50)))
fig = plt.gcf()
fig.set_size_inches(16,16)

In [ ]:
position = round(shape[1] * 0.60)
image_slice = image[:, position, :]
plt.imshow(clahe(image_slice.squeeze(), tile_size=(10,10)))
fig = plt.gcf()
fig.set_size_inches(16,16)

In [ ]:
neighbor_count = 25
center_pos = round(shape[1] * 0.50)
pos = slice(center_pos - neighbor_count, center_pos + neighbor_count + 1)

In [ ]:
image_slice = image[:, pos, :]
image_slice_mean = image_slice.mean(axis=1)
image_slice_min = image_slice.min(axis=1)
image_slice_std = image_slice.std(axis=1)
#image_slice = np.median(image_slice, axis=1)
print(image_slice_mean.shape)
plt.imshow(image_slice[:, neighbor_count + 1, :].squeeze())
plt.gcf().set_size_inches(16,16)

In [ ]:
from skimage.exposure import histogram

hist, bin_edges = histogram(image_slice_min)
plt.bar(bin_edges, hist)

In [ ]:
plt.plot(image_slice_min[700, ...].squeeze())

In [ ]:
adj_min = clahe(image_slice_min, tile_size=(20,20))
adj_mean = image_slice_mean
#adj = image_slice
plt.imshow(adj)
plt.gcf().set_size_inches(16,16)

In [ ]:
plt.plot(adj[700, ...].squeeze())

In [ ]:
th = np.logical_and(adj_min < 140, adj_mean < 160)
plt.imshow(th)
plt.gcf().set_size_inches(16,16)

In [ ]:
from skimage.morphology import disk, opening

d = disk(2)
morph = opening(th, d)
plt.imshow(morph > 0)
plt.gcf().set_size_inches(16,16)

In [ ]:
from skimage.measure import label

lab = morph.copy()
lab[0, :] = 1
lab[-1, :] = 1
lab[:, 0] = 1
lab[:, -1] = 1
lab = label(lab)
lab[lab == 1] = 0
plt.imshow(lab > 0)
plt.gcf().set_size_inches(16,16)

In [ ]:
bg = image_slice[:, neighbor_count+1, :].squeeze()[..., np.newaxis]
print(bg.shape)
fg = lab.astype(np.uint8)[..., np.newaxis]
print(fg.shape)

ov = overlay(bg, fg, color=[1.0,1.0,0.0], alpha=1.0, beta=1.0)
plt.imshow(ov)
plt.gcf().set_size_inches(16,16)

In [ ]:
temp = np.transpose(image, (2, 0, 1))
position = round(temp.shape[0] * 0.50)
image_slice = temp[position, :, :]
plt.imshow(image_slice.squeeze())
fig = plt.gcf()
fig.set_size_inches(16,16)

In [ ]:
m = montage(temp[[1900,1950,2000,2050],...][..., np.newaxis], maximum_images=4)
plt.imshow(m.squeeze())
fig = plt.gcf()
fig.set_size_inches(16,16)